In [ ]:
import os, sys
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors as colors
from tqdm import tqdm
import bornagain as ba
from bornagain import deg, angstrom, nm

In [ ]:
N_FRAMES = 100

In [ ]:
def run_simulation(xi=0.0):
    # defining materials
    m_air = ba.HomogeneousMaterial("Air", 0.0, 0.0 )
    m_substrate = ba.HomogeneousMaterial("Substrate", 6e-6, 2e-8 )
    m_particle = ba.HomogeneousMaterial("Particle", 6e-4, 2e-8 )
    
    # particle
    cylinder_ff = ba.FormFactorCylinder(10*nm, 5*nm)
    cylinder = ba.Particle(m_particle, cylinder_ff)
    layout = ba.ParticleLayout()
    layout.addParticle(cylinder)

    # interference function
    lattice = ba.SquareLattice(25*nm, xi*deg)
    interference = ba.InterferenceFunction2DLattice(lattice)
    decay = ba.FTDecayFunction2DCauchy(1000*nm, 1000*nm)
    interference.setDecayFunction(decay)
    layout.setInterferenceFunction(interference)

    # layers
    air_layer = ba.Layer(m_air)
    air_layer.addLayout(layout)
    substrate_layer = ba.Layer(m_substrate)

    # multilayer
    multi_layer = ba.MultiLayer()
    multi_layer.addLayer(air_layer)
    multi_layer.addLayer(substrate_layer)

    # build and run experiment
    simulation = ba.GISASSimulation()
    simulation.setDetectorParameters(100, -2.0*deg, 2.0*deg, 100, 0.0*deg, 4.0*deg)
    simulation.setBeamParameters(1.38*angstrom, 0.5*deg, 0.0*deg)
    simulation.setBeamIntensity(1e6)

    # run simulation
    simulation.setSample(multi_layer)
    simulation.runSimulation()
    return simulation.result()

In [ ]:
def calculate_xi(i):
    return 0. + (90.0/N_FRAMES)*i

In [ ]:
fig = plt.figure(figsize=(5,5))
for i in tqdm(range(N_FRAMES)):
    xi = calculate_xi(i)
    result = run_simulation(xi)
    array = np.log(result.array() + 1.0)
    fig.clear()
    im = plt.imshow(array, vmax=6,
             extent=[-4.0, 4.0, 0, 8.0])
    plt.xlabel(r'$\phi_f$', fontsize=20)
    plt.ylabel(r'$\alpha_f$', fontsize=20)
    if i==0:
        plt.colorbar(im)
    fname = '_tmp%03d.png'%i
    fig.savefig(fname)
fig.clear()
os.system("mencoder 'mf://_tmp*.png' -mf type=png:fps=10 -ovc lavc -lavcopts vcodec=wmv2 -oac copy -o animation.mp4")
print("Removing temporary files")
os.system("rm _tmp*");

In [ ]:
!mplayer animation.mp4 -loop 0 > /dev/null 2>&1